In [14]:
HOPSWORKS_PROJECT_NAME = 'dpetrik2'

In [2]:
import os, sys

sys.path.append('c:\\Projects\\taxi_predict\\src\\')

In [5]:
import os 
from dotenv import load_dotenv
from paths import PARENT_DIR

load_dotenv(PARENT_DIR / '.env')
HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [6]:
from datetime import datetime
import pandas as pd
from data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    #download data for whole year
    rides_one_year = load_raw_data(year)

    #append rows
    rides = pd.concat([rides, rides_one_year])

File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 was already in local storage
File 2023-08 was already in local storage
File 2023-09 was already in local storage
File 2023-10 was already in local storage
File 2023-11 was already in local storage
File 2023-12 was already in local storage
2024-01 file is not available
2024-02 file is not available
2024-03 file is not available
2024-04 file is not available
2024-05 file is not available
2024-06 file is not available
2024-07 file is not available
2024-08 file is not available
2024-09 file is not available
2024-10 file is not available
2024-11 file is not available
2024-12 file is not available


KeyError: "None of [Index(['pickup_datetime', 'pickup_location_id'], dtype='object')] are in the [columns]"

In [7]:
print(f'{len(rides)=}')

len(rides)=77965659


In [9]:
from data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:09<00:00, 27.46it/s]


In [12]:
import hopsworks

c:\Users\dpetrik\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549012


In [16]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [17]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [18]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description='Time-series data at hourly frequency',
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour'
)

In [20]:
#save data to it

feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 4642800/4642800 | Elapsed Time: 05:01 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549012/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x13882a085b0>, None)